In [ ]:
#STEP 1- Scraping WikiPedia and Creating Data Frame

# install and import libraries
#neighbourhood maps from open.toronto.ca


# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install folium
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install html5lib 

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import folium
print('DONE!')

# Get the dataset metadata by passing package_id to the package_search endpoint
# For example, to retrieve the metadata for this dataset:

url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "4def3f65-2a65-4a4f-83c4-b2a4aed72d46"}
package = requests.get(url, params = params).json()
#print(package["result"])

In [121]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url)
postal_code_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source.text, 'html')
soup1 = BeautifulSoup(postal_code_url,'lxml')
#print(soup.tbody.prettify())
#print("VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV")
#print(soup.tbody.prettify())

#using soup object, iterate the wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find("tbody")

#My_table = soup.find('tbody')
#df = pd.read_html(str(My_table))[0]
#df
#print(df)

#function for finding italics 
def has_italics_not_assigned(tag):
    return tag.has_attr('i') and not tag.has_attr('a')

def extract_postal(pcode):
    return pcode[0:3]

def extract_borough(borough):
    borough = borough[3:len(borough)]
    borough = borough.rpartition("(")[0]
    return borough

def extract_neighbourhood(nbhood):
    nbhood = nbhood.rpartition("(")[2]
    nbhood = nbhood.rpartition(")")[0]
    return nbhood


for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
            td_string = td.text.strip()
            borough = td_string[3:len(td_string)]
            
            if (borough != "Not assigned"):
                section.append(extract_postal(td_string) + " " + borough)
            else:
                section.append("")
    #First row of data is the header
    if (index == 0):
        columns = ['M1_', 'M2_', 'M3_', 'M4_', 'M5_', 'M6_', 'M7_', 'M8_', 'M9_']
    else:
        data.append(section)

#convert list into Pandas DataFrame
Toronto_df = pd.DataFrame(data = data, columns = columns)
Toronto_df

#Toronto_df.shape
#Toronto_df = Toronto_df[Toronto_df['Borough'] != 'Not assigned']
#Toronto_df.reset_index(inplace=True)
#Toronto_df.drop('index', axis=1, inplace=True)
#print('This table has {} rows and {} columns.'.format(Toronto_df.shape[0], Toronto_df.shape[1]))

#Toronto_df

,M1_,M2_,M3_,M4_,M5_,M6_,M7_,M8_,M9_
0,M1B Scarborough(Malvern / Rouge),,M3B North York(Don Mills)North,M4B East York(Parkview Hill / Woodbine Gardens),"M5B Downtown Toronto(Garden District, Ryerson)",M6B North York(Glencairn),,,M9B Etobicoke(West Deane Park / Princess Garde...
1,M1C Scarborough(Rouge Hill / Port Union / High...,,M3C North York(Don Mills)South(Flemingdon Park),M4C East York(Woodbine Heights),M5C Downtown Toronto(St. James Town),M6C York(Humewood-Cedarvale),,,M9C Etobicoke(Eringate / Bloordale Gardens / O...
2,M1E Scarborough(Guildwood / Morningside / West...,,,M4E East Toronto(The Beaches),M5E Downtown Toronto(Berczy Park),M6E York(Caledonia-Fairbanks),,,
3,M1G Scarborough(Woburn),,,M4G East York(Leaside),M5G Downtown Toronto(Central Bay Street),M6G Downtown Toronto(Christie),,,
4,M1H Scarborough(Cedarbrae),M2H North York(Hillcrest Village),M3H North York(Bathurst Manor / Wilson Heights...,M4H East York(Thorncliffe Park),M5H Downtown Toronto(Richmond / Adelaide / King),M6H West Toronto(Dufferin / Dovercourt Village),,,
5,M1J Scarborough(Scarborough Village),M2J North York(Fairview / Henry Farm / Oriole),M3J North York(Northwood Park / York University),M4J East YorkEast Toronto(The Danforth East),M5J Downtown Toronto(Harbourfront East / Union...,M6J West Toronto(Little Portugal / Trinity),,,
6,M1K Scarborough(Kennedy Park / Ionview / East ...,M2K North York(Bayview Village),M3K North York(Downsview)East (CFB Toronto),M4K East Toronto(The Danforth West / Riverdale),M5K Downtown Toronto(Toronto Dominion Centre /...,M6K West Toronto(Brockton / Parkdale Village /...,,,
7,M1L Scarborough(Golden Mile / Clairlea / Oakri...,M2L North York(York Mills / Silver Hills),M3L North York(Downsview)West,M4L East Toronto(India Bazaar / The Beaches West),M5L Downtown Toronto(Commerce Court / Victoria...,M6L North York(North Park / Maple Leaf Park / ...,,,M9L North York(Humber Summit)
8,M1M Scarborough(Cliffside / Cliffcrest / Scarb...,M2M North York(Willowdale / Newtonbrook),M3M North York(Downsview)Central,M4M East Toronto(Studio District),M5M North York(Bedford Park / Lawrence Manor E...,M6M York(Del Ray / Mount Dennis / Keelsdale an...,,,M9M North York(Humberlea / Emery)
9,M1N Scarborough(Birch Cliff / Cliffside West),M2N North York(Willowdale)South,M3N North York(Downsview)Northwest,M4N Central Toronto(Lawrence Park),M5N Central Toronto(Roselawn),M6N York(Runnymede / The Junction North),,,M9N York(Weston)


In [ ]:
# hi this is an edit hello# what about now? Interesting...